In [ ]:
# 출처 : https://wikidocs.net/159467

### 1.Import(관련내용 설치)

In [1]:
!pip install sentence_transformers -qqq
!pip install konlpy -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 19.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import itertools
import pandas as pd

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [3]:
def listToString(s):  
    str1 = ""  
    for ele in s:  
        str1 += " " + ele.strip()  
    return str1

In [4]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

### 2.Data Load & Text Cleansing

In [17]:
# Data Load
df = pd.read_excel('./20230320_review_data5_temp.xlsx',  1)

In [15]:
df.tail(2)

,Unnamed: 0,GROUP2,KEYWORD,리뷰상세내용,구매자평점,포토/영상,등록자,리뷰등록일1,상품번호,상품명,product_nm,token,token_attribution2,lenght
680,191900,판촉,가격,가격도 좋고 좋아요,5,https://phinf.pstatic.net/checkout.phinf/20230...,ks****,2023-03-14,4684117462,"아몬드브리즈 뉴트리플러스 프로틴 190ml 24팩 [리뷰1,000원]",멸균 아몬드브리즈 [ 프로틴 ] [ 190ml ],가격 도 좋다,가격/Noun 도/Josa 좋다/Adjective,3
681,191966,판촉,가격,가격도 좋고 좋아요,5,https://phinf.pstatic.net/checkout.phinf/20230...,ks****,2023-03-14,5331959810,"아몬드브리즈 뉴트리플러스 식이섬유 190ml 24팩 [리뷰2,000원]",멸균 아몬드브리즈 [ 식이섬유 ] [ 190ml ],가격 도 좋다,가격/Noun 도/Josa 좋다/Adjective,3


In [7]:
# 유형 확인
df.GROUP2.unique()

array(['판촉'], dtype=object)

In [8]:
len(df)

682

In [9]:
# Text Cleansing
doc = listToString(df.REVIEW.sample(frac = 1))

AttributeError: ignored

In [ ]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if (word[1] == 'Noun') | (word[1] == 'Verb') | (word[1] == 'Adjective')])
# tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if (word[1] == 'Noun') ])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

In [ ]:
# 2음절 ~ 3음절
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 10개만 출력 :',candidates[0:9])

### 3.KeyBERT : 기본

In [ ]:
# model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [ ]:
# top_n = 20
# distances = cosine_similarity(doc_embedding, candidate_embeddings)
# keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
# keywords

In [ ]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords

### 4.KeyBERT : Max Sum Similarity

In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
# max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=20, nr_candidates=5)

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=10, nr_candidates=10)

['아이 좋아해요 간식',
 '좋아해요 간식 좋아요',
 '녹아요 매일 먹어요',
 '좋아해요 매일 먹어도',
 '좋아해요 먹어요 받았습니다',
 '좋아합니다 매번 먹고있어요',
 '최고 좋아해요 간식',
 '좋아해요 만족하고 먹여요',
 '사줬더니 좋아해요 먹고있고',
 '좋아해요 매일 사먹어요']

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=20, nr_candidates=20)

['먹고 좋아해요 매번',
 '좋아해요 샀어요 매일',
 '좋아해요 애기 먹고있어요',
 '좋아해요 저렴하게 샀어요',
 '먹구있어요 아이 좋아해요',
 '먹고 좋아해요 최고',
 '좋아하십니다 꾸준히 먹이고있습니다',
 '좋아해요 어린이집 간식',
 '좋아하고 항상 먹에요',
 '좋아해요 먹으라고 시켰어요',
 '아이 좋아해요 간식',
 '좋아해요 간식 좋아요',
 '녹아요 매일 먹어요',
 '좋아해요 매일 먹어도',
 '좋아해요 먹어요 받았습니다',
 '좋아합니다 매번 먹고있어요',
 '최고 좋아해요 간식',
 '좋아해요 만족하고 먹여요',
 '사줬더니 좋아해요 먹고있고',
 '좋아해요 매일 사먹어요']

In [ ]:
# max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=20, nr_candidates=30)

In [ ]:
# max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=7, nr_candidates=40)

In [ ]:
# max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=7, nr_candidates=50)

In [ ]:
# max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=7, nr_candidates=60)

In [ ]:
# max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=7, nr_candidates=70)

In [ ]:
# max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=7, nr_candidates=80)

### 5.KeyBERT : Maximal Marginal Relevance

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=20, diversity=0.1)

['좋아해요 매일 사먹어요',
 '사줬더니 좋아해요 먹고있고',
 '좋아해요 만족하고 먹여요',
 '좋아합니다 매번 먹고있어요',
 '최고 좋아해요 간식',
 '좋아해요 먹어요 받았습니다',
 '녹아요 매일 먹어요',
 '좋아해요 매일 먹어도',
 '좋아해요 간식 좋아요',
 '아이 좋아해요 간식',
 '좋아해요 먹으라고 시켰어요',
 '좋아해요 어린이집 간식',
 '좋아하고 항상 먹에요',
 '좋아하십니다 꾸준히 먹이고있습니다',
 '먹구있어요 아이 좋아해요',
 '먹고 좋아해요 최고',
 '좋아해요 저렴하게 샀어요',
 '좋아해요 샀어요 매일',
 '좋아해요 애기 먹고있어요',
 '좋아해요 매번 만족하며']

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=20, diversity=0.2)

['좋아해요 매일 사먹어요',
 '사줬더니 좋아해요 먹고있고',
 '좋아해요 만족하고 먹여요',
 '좋아합니다 매번 먹고있어요',
 '최고 좋아해요 간식',
 '녹아요 매일 먹어요',
 '좋아해요 먹어요 받았습니다',
 '좋아해요 매일 먹어도',
 '좋아해요 간식 좋아요',
 '좋아해요 먹으라고 시켰어요',
 '좋아해요 어린이집 간식',
 '아이 좋아해요 간식',
 '좋아하고 항상 먹에요',
 '먹구있어요 아이 좋아해요',
 '좋아하십니다 꾸준히 먹이고있습니다',
 '좋아해요 저렴하게 샀어요',
 '먹고 좋아해요 최고',
 '좋아해요 샀어요 매일',
 '좋아해요 매번 만족하며',
 '좋아해요 애기 먹고있어요']

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=20, diversity=0.5)

['좋아해요 매일 사먹어요',
 '농축 토마토 남편',
 '아기 먹는 분유',
 '스마트 스토어 구매',
 '가장 좋아하는 파스타',
 '치즈 좋아해요 요미',
 '가족 먹는 제품',
 '요미 매번 구매',
 '사업 번창 하세요',
 '좋아해요 이유식 먹다',
 '제품 믿고 재구매',
 '매일유업 분유 최고',
 '요미 모두 좋아해요',
 '아기 좋아해요 유기농',
 '봉사 비싸지만 인터넷',
 '좋아하는 제품 구매',
 '간식 좋아요 임신',
 '먹이는 매일 명작',
 '먹이는 아이 좋아해요',
 '많을 유용한 먹거리']

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=20, diversity=0.8)

['좋아해요 매일 사먹어요',
 '분유 거부',
 '야채 농축 토마토',
 '영화관 광고',
 '철분 복용',
 '동물 장난감',
 '인터넷 대량 구매',
 '츠바 중독',
 '부모님 등산 하실',
 '건강 위해 투자',
 '공부 하다',
 '식품 매달 박스',
 '발렌타인 데이',
 '네이버 스토어',
 '가장 좋아하는 파스타',
 '프로 렉스 스포츠',
 '검은 효능 생각',
 '노랑 치즈',
 '출근 하기전',
 '팩홀더때매 요미 요미']

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=20, diversity=0.9)

['좋아해요 매일 사먹어요',
 '분유 거부',
 '이즈 아이',
 '영화관 광고',
 '농축 토마토 남편',
 '공부 하다',
 '철분 흡수',
 '인터넷 대량 구매',
 '부모님 등산 하실',
 '킨더 장난감 동물',
 '발렌타인 데이',
 '식품 매달 박스',
 '스포츠 코드 링크',
 '출근 하기전',
 '검은 효과',
 '노랑 치즈',
 '새로운 음료',
 '가장 좋아하는 파스타',
 '혈당 관리',
 '리즈 무당']

### 9.KoGPT-2 : 문장 생성

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Downloading:   0%|          | 0.00/0.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/490M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'lm_head.weight', 'transformer.h.10.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.7.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [ ]:
sent = '오트 음료'

input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)


tf.Tensor([[ 9114  8599 34398]], shape=(1, 3), dtype=int32)


In [ ]:
output = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        use_cache=True)
output_ids = output.numpy().tolist()[0]
print(output_ids)

[9114, 8599, 34398, 7055, 20237, 7470, 17624, 7532, 9508, 8146, 7312, 7162, 739, 8430, 8548, 8149, 9185, 41190, 9276, 15887, 9136, 9314, 28112, 9109, 7892, 9025, 9029, 7335, 8704, 12032, 8236, 9036, 10623, 8137, 33032, 10599, 14441, 7502, 10090, 21000, 9095, 10023, 14377, 12401, 10433, 24177, 9788, 9688, 10278, 10261, 9271, 38348, 15882, 9341, 10967, 9717, 19519, 14908, 11686, 9330, 11258, 35943, 9098, 7431, 9173, 29749, 9120, 24625, 14411, 10721, 11459, 9023, 15859, 22717, 10574, 29842, 9080, 22447, 9017, 26421, 10869, 9359, 11999, 12976, 13078, 9172, 7851, 46989, 22508, 13444, 11998, 9582, 9515, 9810, 9859, 9796, 49678, 16233, 47252, 20247, 25706, 22921, 25679, 10206, 14643, 9564, 34166, 13358, 10846, 10635, 9745, 9346, 21709, 15918, 19453, 20662, 17764, 9033, 9199, 15177, 45786, 8135, 9351, 16709, 24873, 7198, 9391, 15787]


In [ ]:
tokenizer.decode(output_ids)

'오트 음료나 커피를 마시면 된다.\n이때는 칵테일이나 와인 등을 곁들이거나 가볍게 마실 수 있다.\n또한 맥주 한 잔을 마시는 것도 좋다.\n맥주는 알코올 도수가 낮아 부담 없이 즐길 수도 있고, 와인은 달콤한 맛이 있어 여성들에게 인기가 높다.\n하지만 음주가 건강에 해롭다는 인식이 강해지면서 최근에는 술자리에서 술을 즐기는 사람들이 늘고 있는 추세다.\n특히 최근 들어 젊은 층 사이에서 ‘술 마신다’는 말이 유행처럼 번지고 있는데, 이는 술이 몸에 좋다는 잘못된 인식을 심어주기 때문이다.\n술을 많이 마시고 싶다면 먼저 자신의 체질에 맞는 적당한 양을 섭취하는 것이 중요하다.\n알코올은 우리 몸의 신진대사를 방해'

In [ ]:
"""# 2. Numpy로 Top 5 뽑기"""

import numpy as np

output = model(input_ids)

output.logits

output.logits.shape

top5 = tf.math.top_k(output.logits[0, -1], k=5)

tokenizer.convert_ids_to_tokens(top5.indices.numpy())


['나', '▁등', '수', '인', '에']

In [ ]:
# sent = '근육이 커지기 위해서는'
sent = '어메이징 오트 식물 기름 냄새'
input_ids = tokenizer.encode(sent)

import random

while len(input_ids) < 50:
    output = model(np.array([input_ids]))
    top5 = tf.math.top_k(output.logits[0, -1], k=5)
    token_id = random.choice(top5.indices.numpy())
    input_ids.append(token_id)

tokenizer.decode(input_ids)

'어메이징 오트 식물 기름 냄새 나는 찻잎의 향은 그 어떤 향기일까요?\n그런 향기라면 뭐라고 생각하겠나요.\n아 그래?\n예~ 아~ 이~ 향을 음음 어떨까?\n그런 느낌이 들어'